In [10]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

from skimage import data, transform, io, exposure
from skimage.color import rgb2gray
from skimage.feature import match_descriptors, plot_matches, SIFT, match_template, ORB
from skimage.transform import rescale, resize, downscale_local_mean, estimate_transform,  warp, AffineTransform

import rasterio as rio
from rasterio.crs import CRS

import cv2
from affine import Affine
import numpy as np
import os

Image.MAX_IMAGE_PIXELS = 933120000

In [11]:
# REFERENCE IMAGE FROM QGIS
ref_image = Image.open(r"data\GIS\Harris_full.tiff")

# TILE INDEX TO TEST ON
test_index = Image.open(r"data\000_WorkingFiles\HarrisIndexes\48201CIND0_0992.tif")

In [22]:
def preprocess(image):
    image = np.asarray(image) * 255
    image = image.astype(np.uint8)
    image = cv2.GaussianBlur(image, (5, 5), 0)
    image = np.where(image < 100, image, 255).astype(np.uint8)
    # image = cv2.dilate(image, np.ones((3,3), np.uint8), iterations=1).astype(np.uint8)
    return image

test_preprocess = preprocess(test_index)
Image.fromarray(test_preprocess).save("test.png")

In [23]:
# Create the SIFT object
sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors for the images
keypoints1, descriptors1 = sift.detectAndCompute(np.array(ref_image), None)
keypoints2, descriptors2 = sift.detectAndCompute(test_preprocess, None)

In [24]:
# Draw keypoints on the images
image1_keypoints = cv2.drawKeypoints(np.array(ref_image), keypoints1, None)
image2_keypoints = cv2.drawKeypoints(test_preprocess, keypoints2, None)

In [25]:
def save(image, name):
    Image.fromarray(image).save(name)
    
save(image1_keypoints,"ref.png")
save(image2_keypoints,"tile.png")

In [26]:
# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 50)
search_params = dict(checks=50) # or pass empty dictionary
flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(descriptors1,descriptors2,k=2)

In [27]:
# Need to draw only good matches, so create a mask
matchesMask = [[0,0] for i in range(len(matches))]
# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.7*n.distance:
        matchesMask[i]=[1,0]

draw_params = dict(matchColor = (0,255,0),
    singlePointColor = (255,0,0),
    matchesMask = matchesMask,
    flags = cv2.DrawMatchesFlags_DEFAULT)


img3 = cv2.drawMatchesKnn(np.array(ref_image),
                         keypoints1,
                         test_preprocess,
                         keypoints2,
                         matches,
                         None,**draw_params)

In [28]:
save(img3, "testout.png")